In [1]:
import datetime as dt
from collections import namedtuple
import pandas as pd

In [2]:
InputLine = namedtuple("InputLine", ["date", "guard", "asleep"])

file = "input.txt"
with open(file, "r") as f:
    lines = []
    for line in f.readlines():
        cleaned = line.split("] ")
        date_str = cleaned[0].replace("[", "")
        date = dt.datetime.strptime(date_str, "%Y-%m-%d %H:%M")
        date = date.replace(year= date.year + 500)
        text = cleaned[1]
        if text.startswith("Guard"):
            guard = int(text.replace("Guard #", "").replace(" begins shift", ""))
        else:
            guard = None
        if text.startswith("falls asleep"):
            asleep = True
        else:
            asleep = False
        entry = InputLine(date=date, guard=guard, asleep=asleep)
        lines.append(entry)

lines.sort()


In [3]:
df = pd.DataFrame(lines).set_index("date").sort_index()

In [4]:
df.index = pd.to_datetime(df.index)

In [5]:
new_index = pd.date_range(df.index.min(), df.index.max().replace(minute=59), freq="T")

In [6]:
reindex = df.reindex(new_index)

In [7]:
drop_hours = reindex.fillna(method="ffill").loc[reindex.index.hour < 1].copy()

In [8]:
max_guard = int(drop_hours.groupby("guard").sum().idxmax())

In [9]:
find_minute = drop_hours.reset_index().loc[lambda anything: anything["guard"] == max_guard]
find_minute["minute"] = find_minute["index"].dt.minute
max_minute = int(find_minute.groupby("minute").agg({"asleep": "sum"}).idxmax())

In [10]:
answer = max_guard * max_minute

In [11]:
answer

102688

In [12]:
df = drop_hours.copy()

In [13]:
df['minute'] = df.index.minute
df['ones'] = 1
df = df.loc[df["asleep"]].copy()

In [14]:
df.head()

,guard,asleep,minute,ones
2018-03-03 00:42:00,947.0,True,42,1
2018-03-03 00:43:00,947.0,True,43,1
2018-03-03 00:44:00,947.0,True,44,1
2018-03-03 00:45:00,947.0,True,45,1
2018-03-03 00:46:00,947.0,True,46,1


In [15]:
piv = pd.pivot_table(df, index="minute", columns="guard", values="ones", aggfunc="count").fillna(0)

In [16]:
piv.max()

guard
113.0      8.0
389.0     15.0
409.0      6.0
449.0     14.0
557.0      9.0
601.0      4.0
739.0      6.0
877.0     11.0
947.0     12.0
1033.0     6.0
1237.0     5.0
1307.0    11.0
1459.0    18.0
1559.0    11.0
1811.0     9.0
2647.0    10.0
3067.0    10.0
3209.0    14.0
3217.0    14.0
3271.0    14.0
dtype: float64

In [17]:
max_guard = piv.max().idxmax()

In [18]:
piv.idxmax()

guard
113.0     35
389.0     46
409.0     44
449.0     45
557.0     22
601.0     36
739.0     46
877.0     29
947.0     24
1033.0    17
1237.0    52
1307.0    40
1459.0    39
1559.0    29
1811.0    50
2647.0    41
3067.0    29
3209.0    32
3217.0    23
3271.0    32
dtype: int64

In [19]:
max_minute = piv.idxmax().loc[max_guard]

In [20]:
answer = max_guard * max_minute

In [21]:
answer

56901.0